In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###

    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")

    ## 2.
    tag_list = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_list.append(a["href"])

    return tag_list

In [3]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    if sid != 106: #스포츠 뉴스가 아님
      ## 1.
      url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
      "#&date=%2000:00:00&page={page}"
      html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
      "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
      "Chrome/110.0.0.0 Safari/537.36"})
      soup = BeautifulSoup(html.text, "lxml")
      a_tag = soup.find_all("a")

      ## 2.
      tag_list = []
      for a in a_tag:
          if "href" in a.attrs:  # href가 있는것만 고르는 것
              if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                  tag_list.append(a["href"])

      return tag_list

    else:
      url = f"https://sports.news.naver.com"
      html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
      "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
      "Chrome/110.0.0.0 Safari/537.36"})
      soup = BeautifulSoup(html.text, "lxml")
      a_tag = soup.find_all("a")

      ## 2.
      tag_list = []
      for a in a_tag:
          if "href" in a.attrs:  # href가 있는것만 고르는 것
              if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                  tag_list.append(a["href"])

      return tag_list

In [4]:
def re_tag(sid):
    ### 특정 분야의 10페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    re_lst = []
    for i in tqdm(range(10)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)

    return re_lst

In [5]:
all_hrefs = {}
sids = [i for i in range(100,107)]  # 분야 리스트

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수

    Args:
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스

    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리

    """
    art_dict = {}

    ## 1.
    text_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"

    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")

    ## 2.
    # 제목 수집
    text = soup.select(text_selector)
    text_list = [t.text for t in text]
    text_str = "".join(text_list)

    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)

    ## 3.
    art_dict["text"] = text_str
    art_dict["date"] = date_str

    return art_dict

In [8]:
#섹션별 재라벨링
d = {100: "정치", 101: "경제", 102: "사회", 103: "생활/문화", 104: "세계", 105: "IT/과학", 106: "스포츠"}
d2 = {"정치": 6, "경제": 1, "사회": 2, "생활/문화": 3, "세계": 4, "IT/과학": 0, "스포츠": 5}

In [14]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_list = [s for s in range(100, 107)]
artdict_list = []

for section in tqdm(section_list):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dict = art_crawl(all_hrefs, section, i)
        art_dict["ID"] = f"crawling_{i:05d}"  # ID : crawling_00000
        art_dict["target"] = d2[d[section]]
        art_dict["url"] = all_hrefs[section][i]
        artdict_list.append(art_dict)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [17]:
artdict_list

[{'text': '이재명 "윤 대통령, 영부인 의혹 은폐·노골적 당무 개입, 선거 개입에 앞장"',
  'date': '2024.01.26. 오전 9:59',
  'ID': 'crawling_00000',
  'target': 6,
  'url': 'https://n.news.naver.com/mnews/article/087/0001022308?sid=100'},
 {'text': '尹대통령 "정치인 테러는 국민 테러"…정무수석, 배현진 병문안',
  'date': '2024.01.26. 오전 11:26',
  'ID': 'crawling_00001',
  'target': 6,
  'url': 'https://n.news.naver.com/mnews/article/008/0004991167?sid=100'},
 {'text': '한동훈 잘한다 52%…"2012년 박근혜 비대위 같다" [갤럽]',
  'date': '2024.01.26. 오후 1:29',
  'ID': 'crawling_00002',
  'target': 6,
  'url': 'https://n.news.naver.com/mnews/article/015/0004941586?sid=100'},
 {'text': '與, 배현진 피습에 “증오정치 끝내야” 한목소리…29일 대책 논의(종합)',
  'date': '2024.01.26. 오후 12:00',
  'ID': 'crawling_00003',
  'target': 6,
  'url': 'https://n.news.naver.com/mnews/article/018/0005662169?sid=100'},
 {'text': '태영호 "김건희에 명품백 건넨 목사는 친북 활동가…총선용 정치공작"',
  'date': '2024.01.26. 오전 11:19',
  'ID': 'crawling_00004',
  'target': 6,
  'url': 'https://n.news.naver.com/mnews/article/088/0000859124?sid=

In [16]:
art_df = pd.DataFrame(artdict_list)[['ID','text','target','url','date']]
art_df

,ID,text,target,url,date
0,crawling_00000,"이재명 ""윤 대통령, 영부인 의혹 은폐·노골적 당무 개입, 선거 개입에 앞장""",6,https://n.news.naver.com/mnews/article/087/000...,2024.01.26. 오전 9:59
1,crawling_00001,"尹대통령 ""정치인 테러는 국민 테러""…정무수석, 배현진 병문안",6,https://n.news.naver.com/mnews/article/008/000...,2024.01.26. 오전 11:26
2,crawling_00002,"한동훈 잘한다 52%…""2012년 박근혜 비대위 같다"" [갤럽]",6,https://n.news.naver.com/mnews/article/015/000...,2024.01.26. 오후 1:29
3,crawling_00003,"與, 배현진 피습에 “증오정치 끝내야” 한목소리…29일 대책 논의(종합)",6,https://n.news.naver.com/mnews/article/018/000...,2024.01.26. 오후 12:00
4,crawling_00004,"태영호 ""김건희에 명품백 건넨 목사는 친북 활동가…총선용 정치공작""",6,https://n.news.naver.com/mnews/article/088/000...,2024.01.26. 오전 11:19
...,...,...,...,...,...
484,crawling_00058,"""예산 없어"" 美 제안 거절...우주업계 ""기회 놓치다니"" 비판 [지금이뉴스]",0,https://n.news.naver.com/mnews/article/052/000...,2024.01.26. 오전 9:38
485,crawling_00059,"네이버, 동영상 라이브 플랫폼으로 B2B 시장 겨냥",0,https://n.news.naver.com/mnews/article/277/000...,2024.01.26. 오전 9:11
486,crawling_00060,인류 첫 지구 아닌 행성 비행 화성탐사 '인저뉴이티' 역사속으로,0,https://n.news.naver.com/mnews/article/584/000...,2024.01.26. 오후 1:25
487,crawling_00061,"삼성스토어, AI 제품 원스탑 쇼핑 경험 제공…갤럭시S24부터 스마트싱스 가전까지",0,https://n.news.naver.com/mnews/article/014/000...,2024.01.26. 오후 1:48


In [18]:
art_df.to_csv("crawling_data.csv", index=False)